In [2]:
import sys
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import contextlib
import io

# Import relevant classes from modules
try:
    from CW_Preprocessing import PreprocessingManager
    from Genres import GenreStatisticsAnalyzer
    from Artist import ArtistPopularityAnalyzer
    from Top5 import Top5ArtistsAnalyzer

    print("✅ Modules and classes imported successfully!")

except ImportError as e:
    print(f"❌ ImportError: {e}")
    print("🔍 Check if the module exists and contains the correct function.")

# Initialize database manager
DATABASE_PATH = "CWDatabase.db"

# Instantiate analyzer objects
preprocessing_manager = PreprocessingManager(DATABASE_PATH)
genre_analyzer = GenreStatisticsAnalyzer(DATABASE_PATH)
artist_analyzer = ArtistPopularityAnalyzer(DATABASE_PATH)
top5_analyzer = Top5ArtistsAnalyzer(DATABASE_PATH)

# Global Output Widget
output = widgets.Output(layout={'border': '1px solid black', 'padding': '5px', 'overflow_y': 'auto'})

# Define Complementary Color for Buttons & Tabs
complementary_color = "#FFA500"

# Header Styling with **underline**
header = widgets.HTML("<h2 style='text-align: center; color: black; background: #FFA500; padding: 10px; border-radius: 5px;'><u>🎵 Song Dataset Analyser 🎵</u></h2>")

# 📖 **Collapsible Help Guide**
help_text = widgets.HTML("""
    <h3>📖 Tool Guide</h3>
    <b>🗂 Upload & Select CSV File:</b> Upload a new dataset or select an existing CSV file.<br>
    <b>📂 Update Processing File:</b> Processes and integrates the selected CSV file into the system.<br>
    <b>📈 Genre Analysis:</b> Analyze the most popular genres for a selected year.<br>
    <b>🎤 Artist Popularity:</b> Retrieve statistics for a specific artist.<br>
    <b>🏆 Top 5 Artists:</b> Find the top 5 artists in a given time range.<br>
""")

help_toggle = widgets.Accordion(children=[help_text])
help_toggle.set_title(0, "📖 Tool Guide")
help_toggle.selected_index = None  # Default to collapsed

# File Upload Widget
file_upload = widgets.FileUpload(accept=".csv", multiple=False)

# Function to update dropdown with CSV files from the directory
def update_csv_dropdown():
    csv_files = [f for f in os.listdir(os.getcwd()) if f.endswith(".csv")]
    csv_dropdown.options = ["Select a file..."] + csv_files if csv_files else ["No CSV files found"]

# Dropdown Menu for CSV File Selection
csv_dropdown = widgets.Dropdown(
    options=["Loading..."],
    description="CSV File:",
    layout=widgets.Layout(width="60%")
)

update_csv_dropdown()  # Update dropdown immediately

# File Upload Section
file_upload_section = widgets.VBox([
    widgets.Label("Upload CSV File:"),
    file_upload,
    widgets.Label("Select CSV File:"),
    csv_dropdown
], layout=widgets.Layout(justify_content="center", align_items="center", padding="10px", border="1px solid black"))

# Processing Button
process_button = widgets.Button(
    description="📂 Update Processing File", 
    layout=widgets.Layout(width="300px", height="40px"),
    style={'button_color': complementary_color}
)

# Function to process and integrate new data
def process_and_integrate_data(_):
    with output:
        clear_output(wait=True)
        
        selected_file = csv_dropdown.value
        if selected_file in ["Select a file...", "No CSV files found"]:
            print("⚠ Please select a valid CSV file before processing.")
            return
        
        print(f"🔄 Processing and integrating data from {selected_file}...")

        try:
            data = pd.read_csv(selected_file)
            preprocessing_manager.integrate_new_data(data)
            print(f"✅ Data from {selected_file} successfully integrated into the program!")
        except Exception as e:
            print(f"❌ Error processing and integrating data: {e}")

process_button.on_click(process_and_integrate_data)

# --- Global Input Widgets ---
year_widget = widgets.IntText(description="Year (1998-2020):", layout=widgets.Layout(width="50%"))
start_year_widget = widgets.IntText(description="Start Year:", layout=widgets.Layout(width="40%"))
end_year_widget = widgets.IntText(description="End Year:", layout=widgets.Layout(width="40%"))

# **Artist Input Widgets** (Initially Hidden)
artist_widget = widgets.Text(
    placeholder="Enter artist name...",
    description="Artist:",
    layout=widgets.Layout(width="50%"),
    visible=False  # Initially hidden
)
artist_submit_button = widgets.Button(description="Run Artist Analysis", style={'button_color': complementary_color})

# Submit Buttons
genre_submit_button = widgets.Button(description="Run Genre Analysis", style={'button_color': complementary_color})
top5_submit_button = widgets.Button(description="Run Top 5 Analysis", style={'button_color': complementary_color})

# **Corrected Functions for Running Analyses**
def genre_submit(_):
    with output:
        clear_output(wait=True)
        year = year_widget.value

        if 1998 <= year <= 2020:
            genre_analyzer.analyze_genre_statistics(year)
            display(HTML("<br><i>✅ Green highlighting indicates the genre with the highest value in each category.</i>"))
        else:
            print("⚠ Invalid input. Please enter a year between 1998 and 2020.")

def artist_submit(_):
    with output:
        clear_output(wait=True)  # Clears previous outputs

        artist_name = artist_widget.value.strip()

        if not artist_name:
            print("⚠ Please enter a valid artist name.")
            return

        artist_analyzer.analyze_artist(artist_name)
        display(HTML("<br><i>✅ Green highlighting indicates the artist with the highest popularity score.</i>"))

def top5_submit(_):
    with output:
        clear_output(wait=True)
        start_year = start_year_widget.value
        end_year = end_year_widget.value

        if 1998 <= start_year <= 2020 and 1998 <= end_year <= 2020:
            top5_analyzer.top_5_artists(start_year, end_year)
            display(HTML("<br><i>✅ Green highlighting marks the artist with the highest ranking value.</i>"))
        else:
            print("⚠ Invalid input. Please enter a valid year range between 1998 and 2020.")

# **Function to show the Artist Input box dynamically**
def show_artist_input(_):
    artist_widget.visible = True  # Make artist input visible
    artist_submit_button.visible = True  # Show the button to analyze artist

# Link Buttons to Corrected Functions
genre_submit_button.on_click(genre_submit)
artist_submit_button.on_click(artist_submit)
top5_submit_button.on_click(top5_submit)

# Link the "Run Artist Analysis" button to show artist input
artist_submit_button.on_click(show_artist_input)

# Tabs
tabs = widgets.Tab(children=[
    widgets.VBox([widgets.Label("📈 Genre Analysis: Analyze song genres for a given year."), year_widget, genre_submit_button], layout=widgets.Layout(border="1px solid black", padding="5px")),
    widgets.VBox([widgets.Label("🎤 Artist Popularity: Enter artist name and analyze their data."), artist_widget, artist_submit_button], layout=widgets.Layout(border="1px solid black", padding="5px")),
    widgets.VBox([widgets.Label("🏆 Top 5 Artists: Find the top 5 artists over a time range."), start_year_widget, end_year_widget, top5_submit_button], layout=widgets.Layout(border="1px solid black", padding="5px"))
])
tabs.set_title(0, "Genre Analysis")
tabs.set_title(1, "Artist Analysis")
tabs.set_title(2, "Top 5 Artists")

# Layout
dashboard = widgets.VBox(
    [
        header,
        help_toggle,  # Collapsible Help Guide
        file_upload_section,
        widgets.HBox([process_button], layout=widgets.Layout(justify_content="center", padding="10px")),
        tabs,
        output
    ],
    layout=widgets.Layout(border="2px solid black", padding="20px", border_radius="10px", background="white")
)

# Display final UI
display(dashboard)


✅ Modules and classes imported successfully!
